In [24]:
#Boiler plate
from qiskit import QuantumCircuit, execute, QuantumRegister, ClassicalRegister

#This package will soon be depreciated, but it seems that the hardware
#API has not caught up to the new qiskit-experiments package yet. I got
# an unsupported op error when using the newer version
from qiskit.ignis.verification.tomography import process_tomography_circuits, ProcessTomographyFitter
from qiskit.test.mock import FakeVigo
import qiskit.quantum_info as qi

from numpy import pi

In [3]:
#Load account
from qiskit import IBMQ
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [5]:
#Search for provider
provider = IBMQ.get_provider(hub='ibm-q')
for backend in provider.backends():
    print(backend.status().to_dict())

{'backend_name': 'ibmq_qasm_simulator', 'backend_version': '0.1.547', 'operational': True, 'pending_jobs': 13, 'status_msg': 'active'}
{'backend_name': 'ibmq_armonk', 'backend_version': '2.4.35', 'operational': True, 'pending_jobs': 1, 'status_msg': 'active'}
{'backend_name': 'ibmq_santiago', 'backend_version': '1.4.3', 'operational': True, 'pending_jobs': 85, 'status_msg': 'active'}
{'backend_name': 'ibmq_bogota', 'backend_version': '1.6.41', 'operational': True, 'pending_jobs': 246, 'status_msg': 'active'}
{'backend_name': 'ibmq_lima', 'backend_version': '1.0.36', 'operational': True, 'pending_jobs': 12, 'status_msg': 'active'}
{'backend_name': 'ibmq_belem', 'backend_version': '1.0.42', 'operational': True, 'pending_jobs': 19, 'status_msg': 'active'}
{'backend_name': 'ibmq_quito', 'backend_version': '1.1.30', 'operational': True, 'pending_jobs': 23, 'status_msg': 'active'}
{'backend_name': 'simulator_statevector', 'backend_version': '0.1.547', 'operational': True, 'pending_jobs': 13,

In [16]:
#I picked the one with the fewest jobs, I was only using a single qubit
device = provider.get_backend('ibmq_armonk')

In [17]:
# Put together the process
qr = QuantumRegister(1)
cr = ClassicalRegister(1)
qc = QuantumCircuit(qr,cr)
qc.ry(pi/2, 0)

12


In [27]:
#Qiskit automatically generate the circuits to measure the full set of
#expectation values
qpt_circs = process_tomography_circuits(qc,[0])
for circ in qpt_circs:
    print(circ.name)

(('Zp',), ('X',))
(('Zp',), ('Y',))
(('Zp',), ('Z',))
(('Zm',), ('X',))
(('Zm',), ('Y',))
(('Zm',), ('Z',))
(('Xp',), ('X',))
(('Xp',), ('Y',))
(('Xp',), ('Z',))
(('Yp',), ('X',))
(('Yp',), ('Y',))
(('Yp',), ('Z',))


In [18]:
#run circuits on device
job = execute(qpt_circs, device, shots = 4000)

In [19]:
#Organize data
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs)
qpt_tomo.data

{(('Zp',), ('X',)): {'0': 3910, '1': 90},
 (('Zp',), ('Y',)): {'0': 2120, '1': 1880},
 (('Zp',), ('Z',)): {'0': 2078, '1': 1922},
 (('Zm',), ('X',)): {'0': 236, '1': 3764},
 (('Zm',), ('Y',)): {'0': 2034, '1': 1966},
 (('Zm',), ('Z',)): {'0': 1997, '1': 2003},
 (('Xp',), ('X',)): {'0': 2073, '1': 1927},
 (('Xp',), ('Y',)): {'0': 2042, '1': 1958},
 (('Xp',), ('Z',)): {'0': 237, '1': 3763},
 (('Yp',), ('X',)): {'0': 2077, '1': 1923},
 (('Yp',), ('Y',)): {'0': 3900, '1': 100},
 (('Yp',), ('Z',)): {'0': 2164, '1': 1836}}

In [23]:
#Find the closest TPCP map to the measured choi matrix
choi_fit = qpt_tomo.fit(method='lstsq')
#Convert to Kraus operators
kraus_ops = qi.Kraus(choi_fit)
#get gate fidelity (out of curiosity)
print('Average gate fidelity: F = {:.5f}'.format(qi.average_gate_fidelity(choi_fit, target=qi.Operator(qc))))

Average gate fidelity: F = 0.95365


In [21]:
#Read off Kraus operators
for i,op in enumerate(kraus_ops.data):
    print("Kraus op no. " + str(i) + ":\n")
    print(op)
    print("\n")

Kraus op no. 0:

[[ 0.69656895-1.45919336e-17j -0.67437702-3.12949635e-02j]
 [ 0.68150683+2.14814712e-02j  0.67477499+2.38053192e-02j]]


Kraus op no. 1:

[[ 0.11887707-1.29837972e-11j  0.16558419+4.86069853e-02j]
 [ 0.07790308+6.79040616e-03j -0.03511688+3.52818041e-02j]]


Kraus op no. 2:

[[ 0.14124606-5.47767497e-10j -0.01251611-1.02168682e-01j]
 [-0.08117136+8.19437890e-02j -0.07699419-1.89589605e-01j]]


